### Import

In [197]:
import sys
import time
import pickle
import gzip
from random import randint
from scipy import misc
from scipy import special
import scipy.ndimage
from scipy.sparse import csr_matrix, csc_matrix
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans,MiniBatchKMeans
import matplotlib.pyplot as plt
import json
import csv
import collections
import math
import sys

### Initial Settings

In [2]:
DATA_PATH = 'data/mnist/'

IMAGES_TRAIN = 'data_training'
IMAGES_TEST = 'data_testing'

RANDOM_SEED = 42
N_CLASSES = 10
N_FEATURES = 28 * 28

In [3]:
data_training = DATA_PATH+IMAGES_TRAIN
data_testing = DATA_PATH+IMAGES_TEST
ft = gzip.open(data_training, 'rb')
TRAINING = pickle.load(ft)
ft.close()
ft = gzip.open(data_testing, 'rb')
TESTING = pickle.load(ft)
ft.close()

# Neural Network

In [184]:
class Neural_Network:
    
    def __init__(self, neurons, batchsize, stop_function, stop_parameter):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        self.iteration = 0
        
        # Standardize random weights
        np.random.seed(RANDOM_SEED)
        hidden_layer = np.random.rand(self.neurons, self.input_size + 1) / self.neurons
        output_layer = np.random.rand(self.output_size, self.neurons + 1) / self.output_size
        self.layers = [hidden_layer, output_layer]

    def train(self, training, testing):
        
        accu_train = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nBatch Train: %d\nBatch Test: %d\n%s\n' % (self.neurons,len_batch_train,len_batch_test,typeTrainingPrint))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        test_input = training[0:len_batch_train][0:len_batch_train]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1

        # Performs iterations
        while not self.is_stop_function_enabled(accu_train[1]):
            
            self.iteration += 1
            
            for input_vector, target_vector in zip(inputs, targets):
                self.backpropagate(input_vector, target_vector)
            
            # Accuracy
            accu_test = self.accu(test_output)
            accu_train = self.accu(test_input)
            
            # Messages
            if (self.iteration == 1 or self.iteration % 10 == 0):
                self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                
        # Print last epoch
        if (self.iteration % 10 != 0):
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))

        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector):
        outputs = []
        for layer in self.layers:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = np.inner(layer, input_with_bias)
            output = special.expit(output)
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target):
        c = 10**(-4) + 10**(-1)/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector)

        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        self.layers[-1] -= c*np.outer(output_deltas, np.append(hidden_outputs, 1))

        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(self.layers[-1], self.neurons, 1).T, output_deltas)
        self.layers[0] -= c*np.outer(hidden_deltas, np.append(input_vector, 1))

    def predict(self, input_vector):
        return self.feed_forward(input_vector)[-1]

    def predict_one(self, input_vector):
        return np.argmax(self.feed_forward(input_vector)[-1])

    def accu(self, testing_batch):
        res = np.zeros((10, 2))
        for k in range(len(testing_batch[1])):
            if self.predict_one(testing_batch[0][k]) == testing_batch[1][k]:
                res[testing_batch[1][k]] += 1
            else:
                res[testing_batch[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy TRAIN: '+str(accu_train[1]).zfill(4)+'%\t'
        message += 'Accuracy TEST: '+str(accu_test[1]).zfill(4)+'%\t'
        message += 'Min: '+ str(accu_test[0]).zfill(4)+ '% ('+str(int(accu_test[2]))+')'
        print(message)
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
        
    def getWeights(self):
        return self.layers

In [186]:
nn = Neural_Network(neurons=300,batchsize=1,stop_function=2,stop_parameter=0.01)
nn.train(TRAINING,TESTING)

-- Training Session Start (2019-01-22 16:31:45.401925) --

Neurons: 300
Batch Train: 60000
Batch Test: 10000
Stop Function: improvements below 0.01%

Epoch 001 (42s)    Accuracy TRAIN: 91.11%	Accuracy TEST: 91.53%	Min: 78.59% (5)
Epoch 010 (6m58s)  Accuracy TRAIN: 97.74%	Accuracy TEST: 97.0%	Min: 94.85% (9)
Epoch 020 (14m0s)  Accuracy TRAIN: 98.6%	Accuracy TEST: 97.63%	Min: 96.3% (7)
Epoch 030 (21m8s)  Accuracy TRAIN: 98.93%	Accuracy TEST: 97.78%	Min: 96.63% (9)
Epoch 033 (23m17s) Accuracy TRAIN: 98.99%	Accuracy TEST: 97.8%	Min: 96.63% (9)

-- Training Session End (2019-01-22 16:55:02.655120) --


# NEURAL NETWORK with CLUSTERING

### K-Means

In [6]:
def nearest_centroid_index(centers,value):
    centers = np.asarray(centers)
    idx = (np.abs(centers - value)).argmin()
    return idx

In [7]:
def build_clusters(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(np.hstack(weights).reshape(-1,1))
    return kmeans.cluster_centers_

### Matrix - Helping Function

In [8]:
def redefine_weights(weights,centers):
    arr_ret = np.empty_like(weights).astype(np.int16)
    for i, row in enumerate(weights):
        for j, col in enumerate(row):
            arr_ret[i,j] = nearest_centroid_index(centers,weights[i,j])
    return arr_ret

In [9]:
def idx_matrix_to_matrix(idx_matrix,centers,shape):
    return centers[idx_matrix.reshape(-1,1)].reshape(shape)

In [10]:
def centroid_gradient_matrix(idx_matrix,gradient,cluster):
    return scipy.ndimage.sum(gradient,idx_matrix,index=range(cluster))

### Class

In [254]:
class Neural_Network_KM:

    def __init__(self, neurons, batchsize, cluster, pre_weights, stop_function, stop_parameter):
        
        start_setting_time = dt.datetime.now()
        
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.cluster = cluster
        self.iteration = 0
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        
        # Variable for shape
        shape_hidden = (self.neurons,self.input_size+1)
        shape_output = (self.output_size,self.neurons+1)
        self.layers_shape = [shape_hidden,shape_output]
            
        # Initialize cluster for pre-trained weights (dict with centers)
        c_hidden = build_clusters(self.cluster,pre_weights[0])
        c_output = build_clusters(self.cluster,pre_weights[-1])
        self.centers = [c_hidden,c_output]
        
        # Initialize index matrix for pre-trained weights
        idx_hidden = redefine_weights(pre_weights[0],self.centers[0])
        idx_output = redefine_weights(pre_weights[-1],self.centers[-1])
        self.idx_layers = [idx_hidden,idx_output]
        
        # Setting time print    
        end_setting_time = dt.datetime.now() - start_setting_time
        eta = divmod(end_setting_time.days * 86400 + end_setting_time.seconds, 60)
        self.eta_print_setting = str(eta[0])+"m"+str(eta[1])+"s"
    
 

    def train(self, training, testing):
        
        accu_train = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        test_input = training[0:len_batch_train][0:len_batch_train]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nClusters: %d\nBatch Train: %d (%d%%)\nBatch Test: %d (%d%%)\n%s\n' % (self.neurons,self.cluster,len_batch_train,self.batchsize*100,len_batch_test,self.batchsize*100,typeTrainingPrint))
        
        # Performs iterations
        while not self.is_stop_function_enabled(accu_train[1]):
            
            self.iteration += 1
            
            # Backpropagate with feed forward
            for input_vector, target_vector in zip(inputs, targets):
                weights = []
                for i,c,s in zip(self.idx_layers,self.centers,self.layers_shape):
                    w = idx_matrix_to_matrix(i,c,s)
                    weights.append(w)
                self.backpropagate(input_vector, target_vector, weights)
                
            # Accuracy
            accu_test = self.accu(test_output,weights)
            accu_train = self.accu(test_input,weights)
            
            # Messages
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                      
        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))
        print('-------------------------')
        print(self.printLineCSV(self.cluster,self.ETAepoch(self.start_time),accu_train,accu_test))
        print('-------------------------\n')

    def feed_forward(self, input_vector, weights):
        outputs = []
        for w in weights:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = np.inner(w, input_with_bias)
            output = special.expit(output) # Sigmoid function
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target, weights):
        c = 10**(-4) + (10**(-1))/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector, weights)

        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        gradient = np.outer(output_deltas, np.append(hidden_outputs, 1))
        cg = centroid_gradient_matrix(self.idx_layers[-1],gradient,self.cluster)
        self.centers[-1] = self.centers[-1] - c * np.array(cg).reshape(self.cluster,1)

        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(weights[-1], self.neurons, 1).T, output_deltas)
        gradient = np.outer(hidden_deltas, np.append(input_vector, 1))
        cg = centroid_gradient_matrix(self.idx_layers[0],gradient,self.cluster)
        self.centers[0] = self.centers[0] - c * np.array(cg).reshape(self.cluster,1)
        
    
    
    def predict(self, input_vector, weights):
        return self.feed_forward(input_vector,weights)[-1]

    def predict_one(self, input_vector, weights):
        return np.argmax(self.feed_forward(input_vector,weights)[-1])

    def accu(self, testing, weights):
        res = np.zeros((10, 2))
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k], weights) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy TRAIN: '+str(accu_train[1]).zfill(4)+'%\t'
        message += 'Accuracy TEST: '+str(accu_test[1]).zfill(4)+'%\t'
        message += 'Min: '+ str(accu_test[0]).zfill(4)+ '% ('+str(int(accu_test[2]))+')'
        print(message)
        
    def getWeights(self):
        return self.layers
    
    def minsec2sec(self,time):
        if 'm' in time:
            splitted = time.split('m')
            return int(splitted[0]) * 60 + int(splitted[1][:-1])
        else:
            return int(time[:-1])
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
    
    def printLineCSV(self,cluster,time,a_train,a_test):
        cr = round((784*300)*64/((784*300)*math.log(cluster,2) + cluster*64),3)
        return str(cluster)+','+str(cr)+','+time+','+str(a_train[1])+','+str(a_test[1])

In [189]:
pre_trained_weights = nn.getWeights()
nn_km = Neural_Network_KM(neurons=300,batchsize=1,cluster=256,pre_weights=pre_trained_weights,stop_function=2,stop_parameter=0.01)
nn_km.train(TRAINING,TESTING)

-- Training Session Start (2019-01-22 17:07:57.714760) --

Neurons: 300
Clusters: 256
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.01%

Epoch 001 (3m0s)   Accuracy TRAIN: 98.92%	Accuracy TEST: 97.5%	Min: 95.24% (9)
Epoch 002 (6m0s)   Accuracy TRAIN: 98.93%	Accuracy TEST: 97.48%	Min: 96.23% (9)

-- Training Session End (2019-01-22 17:13:58.625813) --


# NEURAL NETWORK with PRUNING

## CSC - Helping Functions

In [13]:
def pruning_matrix(mat,percentage,method='out'):
    threshold = (100-percentage)
    
    if method == 'inout':
        threshold /= 4
        perc_up,perc_down,perc_mid_up,perc_mid_down = 100 - threshold, threshold, 50 + threshold, 50 - threshold
        percentile_up = np.percentile(mat,perc_up)
        percentile_down = np.percentile(mat,perc_down)
        percentile_mid_up = np.percentile(mat,perc_mid_up)
        percentile_mid_down = np.percentile(mat,perc_mid_down)
    else:
        threshold /= 2
        if method == 'in': perc_up, perc_down = 50 + threshold, 50 - threshold
        elif method == 'out': perc_up, perc_down = 100 - threshold, threshold
        percentile_up = np.percentile(mat,perc_up)
        percentile_down = np.percentile(mat,perc_down)
        
    w_pruned = np.copy(mat)
    for i,row in enumerate(mat):
        for j,_ in enumerate(row):
            if method == 'in':
                if mat[i,j] > percentile_down and mat[i,j] < percentile_up:
                    w_pruned[i,j] = 0
            elif method == 'out':
                if mat[i,j] < percentile_down or mat[i,j] > percentile_up:
                    w_pruned[i,j] = 0
            elif method == 'inout':
                if mat[i,j] < percentile_down or mat[i,j] > percentile_up or (mat[i,j] > percentile_mid_down and mat[i,j] < percentile_mid_up):
                    w_pruned[i,j] = 0
    return w_pruned

In [78]:
'''# in place
def sparse_sub_dense(sparse,dense,mask):
    b = sparse - dense
    b[mask] = 0
    return csc_matrix(b)''' # old

# in place
def sparse_sub_dense(sparse,dense,mask):
    sparse.data -= dense.T[mask.T]

In [15]:
def delete_last_row(csc):
    i = csc.indptr[-1]
    indptr = csc.indptr[:-1]
    data = csc.data[:i]
    indices = csc.indices[:i]
    return csc_matrix((data,indices,indptr))

## Class

In [253]:
class Neural_Network_PR_CSC:
    
    def __init__(self, neurons, batchsize, stop_function, stop_parameter, weights=None, pruning=None, pruning_method=None):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.pre_trained_weights = weights
        self.pruning = pruning
        self.pruning_method = pruning_method
        self.best = 0.
        self.same = 0
        self.iteration = 0
        
        if weights == None:
            # Standardize random weights
            np.random.seed(RANDOM_SEED)
            hidden_layer = np.random.rand(self.neurons, self.input_size + 1) / self.neurons
            output_layer = np.random.rand(self.output_size, self.neurons + 1) / self.output_size
            self.layers = [hidden_layer, output_layer]
        else:
            # Pruning weights
            pw_hidden = csc_matrix(pruning_matrix(pre_trained_weights[0],self.pruning,self.pruning_method))
            pw_output = csc_matrix(pruning_matrix(pre_trained_weights[1],self.pruning,self.pruning_method))
            self.layers = [pw_hidden, pw_output]
            # Matrix mask
            mask_hidden = pw_hidden.A != 0 
            mask_output = pw_output.A != 0
            self.masks = [mask_hidden,mask_output]


    def train(self, training, testing):
        
        accu_train = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nBatch Train: %d (%d%%)\nBatch Test: %d (%d%%)\nPruning: %d%% (%s)\n%s\n' % (self.neurons,len_batch_train,self.batchsize*100,len_batch_test,self.batchsize*100,self.pruning,self.pruning_method,typeTrainingPrint))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        test_input = training[0:len_batch_train][0:len_batch_train]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1

        # Performs iterations
        while not self.is_stop_function_enabled(accu_train[1]):
            
            self.iteration += 1
            
            #print(self.layers[0])
            
            for input_vector, target_vector in zip(inputs, targets):
                self.backpropagate(input_vector, target_vector)
            
            # Accuracy
            accu_test = self.accu(test_output)
            accu_train = self.accu(test_input)
            
            # Messages
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
    
        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))
        print('-------------------------')
        print(self.printLineCSV(self.pruning,self.ETAepoch(self.start_time),accu_train,accu_test))
        print('-------------------------\n')

    def feed_forward(self, input_vector):
        outputs = []
        for layer in self.layers:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = layer * input_with_bias
            output = special.expit(output)
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs

    def backpropagate(self, input_vector, target):
        c = 10**(-4) + (10**(-1))/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector)
        
        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        sparse_sub_dense(self.layers[-1],c*np.outer(output_deltas, np.append(hidden_outputs, 1)),self.masks[-1])
        
        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * (delete_last_row(self.layers[-1]).T * output_deltas)
        sparse_sub_dense(self.layers[0],c*np.outer(hidden_deltas, np.append(input_vector, 1)),self.masks[0])

    def predict(self, input_vector):
        return self.feed_forward(input_vector)[-1]

    def predict_one(self, input_vector):
        return np.argmax(self.feed_forward(input_vector)[-1])

    def accu(self, testing_batch):
        res = np.zeros((10, 2))
        for k in range(len(testing_batch[1])):
            if self.predict_one(testing_batch[0][k]) == testing_batch[1][k]:
                res[testing_batch[1][k]] += 1
            else:
                res[testing_batch[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy TRAIN: '+str(accu_train[1]).zfill(4)+'%\t'
        message += 'Accuracy TEST: '+str(accu_test[1]).zfill(4)+'%\t'
        message += 'Min: '+ str(accu_test[0]).zfill(4)+ '% ('+str(int(accu_test[2]))+')'
        print(message)
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
        
    def getWeights(self):
        return self.layers
    
    def printLineCSV(self,pruning,time,a_train,a_test):
        pr = pruning/100
        cr = round((784*300)/(784*300*pr),3)
        return str(pr)+','+str(cr)+','+time+','+str(a_train[1])+','+str(a_test[1])

In [193]:
pre_trained_weights = nn.getWeights()
nn_pr_csc = Neural_Network_PR_CSC(neurons=300,batchsize=1,weights=pre_trained_weights,pruning=60,pruning_method='in',stop_function=2,stop_parameter=0.01)
nn_pr_csc.train(TRAINING,TESTING)

-- Training Session Start (2019-01-22 17:15:34.146234) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Pruning: 60% (in)
Stop Function: improvements below 0.01%

Epoch 001 (1m54s)  Accuracy TRAIN: 98.82%	Accuracy TEST: 97.56%	Min: 94.94% (7)
Epoch 002 (3m54s)  Accuracy TRAIN: 99.03%	Accuracy TEST: 97.78%	Min: 96.69% (7)
Epoch 003 (5m49s)  Accuracy TRAIN: 99.11%	Accuracy TEST: 97.87%	Min: 96.89% (7)
Epoch 004 (7m43s)  Accuracy TRAIN: 99.17%	Accuracy TEST: 97.92%	Min: 97.08% (7)
Epoch 005 (9m38s)  Accuracy TRAIN: 99.21%	Accuracy TEST: 97.98%	Min: 97.13% (9)
Epoch 006 (11m32s) Accuracy TRAIN: 99.25%	Accuracy TEST: 98.0%	Min: 96.93% (9)
Epoch 007 (13m29s) Accuracy TRAIN: 99.27%	Accuracy TEST: 98.02%	Min: 96.83% (9)
Epoch 008 (15m23s) Accuracy TRAIN: 99.3%	Accuracy TEST: 98.0%	Min: 96.73% (9)
Epoch 009 (17m18s) Accuracy TRAIN: 99.32%	Accuracy TEST: 98.0%	Min: 96.63% (9)
Epoch 010 (19m16s) Accuracy TRAIN: 99.35%	Accuracy TEST: 97.98%	Min: 96.43% (9)
Epoch 011 (21m16s) Acc

# NEURAL NETWORK with PRUNING and CLUSTERING 

## Helper functions

In [18]:
def nearest_centroid_index(centers,value):
    centers = np.asarray(centers)
    idx = (np.abs(centers - value)).argmin()
    return idx

In [19]:
def build_clusters_pruning(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(weights.data.reshape(-1,1))
    return kmeans.cluster_centers_

In [20]:
def redefine_weights_pruning(weights,centers):
    new_data_idx = [nearest_centroid_index(centers,w) for w in weights.data]
    return csc_matrix((new_data_idx,weights.indices,weights.indptr))

In [21]:
def idx_matrix_to_matrix_pruning(idx_matrix,centers):
    return csc_matrix((centers[idx_matrix.data].reshape(-1,),idx_matrix.indices,idx_matrix.indptr))

In [116]:
def centroid_gradient_pruning(idx_matrix,gradient,mask,cluster):
    gradient += 0.000000001
    gradient[mask] = 0
    return scipy.ndimage.sum(csc_matrix(gradient).data,idx_matrix.data,index=range(cluster))

## Class

In [266]:
class Neural_Network_PR_KM:

    def __init__(self, neurons, batchsize, cluster, pruning, pre_weights, stop_function, stop_parameter):
        
        start_setting_time = dt.datetime.now()
        
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.batchsize = batchsize
        self.pruning = pruning
        self.cluster = cluster
        self.iteration = 0
        self.stop_f = stop_function
        self.stop_p = stop_parameter
        self.best = 0.
        self.same = 0
        
        # Matrix mask
        mask_hidden = pre_weights[0].A == 0 
        mask_output = pre_weights[-1].A == 0
        self.masks = [mask_hidden,mask_output]
        
        # Variable for shape
        shape_hidden = (self.neurons,self.input_size+1)
        shape_output = (self.output_size,self.neurons+1)
        self.layers_shape = [shape_hidden,shape_output]
            
        # Initialize cluster for pre-trained weights (dict with centers)
        c_hidden = build_clusters_pruning(self.cluster,pre_weights[0])
        c_output = build_clusters_pruning(self.cluster,pre_weights[-1])
        self.centers = [c_hidden,c_output]
        
        # Initialize index matrix for pre-trained weights
        idx_hidden = redefine_weights_pruning(pre_weights[0],self.centers[0])
        idx_output = redefine_weights_pruning(pre_weights[-1],self.centers[-1])
        self.idx_layers = [idx_hidden,idx_output]
        
        # Setting time print    
        end_setting_time = dt.datetime.now() - start_setting_time
        eta = divmod(end_setting_time.days * 86400 + end_setting_time.seconds, 60)
        self.eta_print_setting = str(eta[0])+"m"+str(eta[1])+"s"
    
 

    def train(self, training, testing):
        
        accu_train = [0.,0.]
        
        # Batch Setting
        len_batch_train = len(training[0])
        len_batch_test = len(testing[0])
        if(self.batchsize > 0 and self.batchsize <= 1):
            len_batch_train = int(np.ceil(len_batch_train * self.batchsize))
            len_batch_test = int(np.ceil(len_batch_test * self.batchsize))
        
        # Divide training and testing batches
        test_output = testing[0:len_batch_test][0:len_batch_test]
        test_input = training[0:len_batch_train][0:len_batch_train]
        inputs = training[0][0:len_batch_train]
        targets = np.zeros((len_batch_train, 10))
        for i in range(len_batch_train):
            targets[i, training[1][i]] = 1
        
        # Start prints 
        self.start_time = dt.datetime.now()
        print('-- Training Session Start (%s) --' % (self.start_time))
        typeTrainingPrint = "Stop Function: "    
        if self.stop_f == 0:
            typeTrainingPrint += str(self.stop_p)+" epochs"
        elif self.stop_f == 1:
            typeTrainingPrint += str(self.stop_p)+" epoch(s) w/o improvements"
        elif self.stop_f == 2:
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        print('\nNeurons: %d\nPruning: %.2f\nClusters: %d\nBatch Train: %d (%d%%)\nBatch Test: %d (%d%%)\n%s\n' % (self.neurons,self.pruning/100,self.cluster,len_batch_train,self.batchsize*100,len_batch_test,self.batchsize*100,typeTrainingPrint))
        
        # Performs iterations
        while not self.is_stop_function_enabled(accu_train[1]):
            
            self.iteration += 1
            
            # Backpropagate with feed forward
            for input_vector, target_vector in zip(inputs, targets):
                weights = []
                for i,c in zip(self.idx_layers,self.centers):
                    w = idx_matrix_to_matrix_pruning(i,c)
                    weights.append(w)
                self.backpropagate(input_vector, target_vector, weights)
                
            # Accuracy
            accu_test = self.accu(test_output,weights)
            accu_train = self.accu(test_input,weights)
            
            # Messages
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                      
        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))
        print('-------------------------')
        print(self.printLineCSV(self.pruning,self.cluster,self.ETAepoch(self.start_time),accu_train,accu_test))
        print('-------------------------\n')

    def feed_forward(self, input_vector, weights):
        outputs = []
        for w in weights:
            input_with_bias = np.append(input_vector, 1)   # Ajout constante
            output = w * input_with_bias
            output = special.expit(output)
            outputs.append(output)
            # The output is the input of the next layer
            input_vector = output
        return outputs                            
                                  
    def backpropagate(self, input_vector, target, weights):
        c = 10**(-4) + (10**(-1))/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector, weights)
        
        # Calculation of partial derivatives for the output layer and subtraction
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        gradient = np.outer(output_deltas, np.append(hidden_outputs, 1))
        cg = centroid_gradient_pruning(self.idx_layers[-1],gradient,self.masks[-1],self.cluster)
        self.centers[-1] = self.centers[-1] - (c * np.array(cg)).reshape(-1,1)
        
        # Calculation of partial derivatives for the hidden layer and subtraction
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * (delete_last_row(weights[-1]).T * output_deltas)
        gradient = np.outer(hidden_deltas, np.append(input_vector, 1))
        cg = centroid_gradient_pruning(self.idx_layers[0],gradient,self.masks[0],self.cluster)
        self.centers[0] = self.centers[0] - (c * np.array(cg)).reshape(-1,1)
        
    
    def predict(self, input_vector, weights):
        return self.feed_forward(input_vector,weights)[-1]

    def predict_one(self, input_vector, weights):
        return np.argmax(self.feed_forward(input_vector,weights)[-1])

    def accu(self, testing, weights):
        res = np.zeros((10, 2))
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k], weights) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        each = [res[k][0]/res[k][1] for k in range(len(res))]
        min_c = sorted(range(len(each)), key=lambda k: each[k])[0]
        return np.round([each[min_c]*100, total[0]/total[1]*100, min_c], 2)
    
    
    def is_stop_function_enabled(self,accuracy):
        if self.stop_f == 0:
            if self.iteration < self.stop_p:
                return False
            else:
                return True
        elif self.stop_f == 1:
            if accuracy > self.best or self.iteration == 0:
                self.same = 0
                self.best = accuracy
                return False
            else:
                self.same += 1
                if self.same < self.stop_p:
                    return False
                else:
                    return True
        elif self.stop_f == 2:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
    
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "
        message += 'Accuracy TRAIN: '+str(accu_train[1]).zfill(4)+'%\t'
        message += 'Accuracy TEST: '+str(accu_test[1]).zfill(4)+'%\t'
        message += 'Min: '+ str(accu_test[0]).zfill(4)+ '% ('+str(int(accu_test[2]))+')'
        print(message)
        
    def getWeights(self):
        return self.idx_layers,self.centers
    
    def minsec2sec(self,time):
        if 'm' in time:
            splitted = time.split('m')
            return int(splitted[0]) * 60 + int(splitted[1][:-1])
        else:
            return int(time[:-1])
    
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
    
    def printLineCSV(self,pruning,cluster,time,a_train,a_test):
        pr = pruning/100
        cr = round((784*300)*64/((784*300*pr)*math.log(cluster,2) + cluster*64),3)
        return str(pr)+','+str(cluster)+','+str(cr)+','+time+','+str(a_train[1])+','+str(a_test[1])

In [248]:
pruned_weights = nn_pr_csc.getWeights()
nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=512,pruning=60,pre_weights=pruned_weights,stop_function=2,stop_parameter=0.01)
nn_pr_km.train(TRAINING,TESTING)

/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-22 18:20:42.356691) --

Neurons: 300
Pruning: 0.60
Clusters: 512
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.01%

Epoch 001 (8m54s)  Accuracy TRAIN: 99.04%	Accuracy TEST: 97.77%	Min: 96.04% (9)
Epoch 002 (17m47s) Accuracy TRAIN: 99.07%	Accuracy TEST: 97.73%	Min: 95.34% (9)
Epoch 003 (26m34s) Accuracy TRAIN: 99.08%	Accuracy TEST: 97.74%	Min: 95.64% (9)

-- Training Session End (2019-01-22 18:47:16.776786) --
-------------------------
0.6,512,11.554,26m34s,97.74,99.08
-------------------------



# EXPERIMENTS

### Pruining e Clustering

In [ ]:
for i in range(70,100,10):
    pre_trained_weights = nn.getWeights()
    nn_pr_csc = Neural_Network_PR_CSC(neurons=300,batchsize=1,weights=pre_trained_weights,pruning=i,pruning_method='in',stop_function=2,stop_parameter=0.02)
    nn_pr_csc.train(TRAINING,TESTING)

    for j in [32,48,64,96,128,192,256,384,512,768,1024,1536,2048]:
        pruned_weights = nn_pr_csc.getWeights()
        nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=j,pruning=i,pre_weights=pruned_weights,stop_function=2,stop_parameter=0.02)
        nn_pr_km.train(TRAINING,TESTING)

In [267]:
pre_trained_weights = nn.getWeights()
nn_pr_csc = Neural_Network_PR_CSC(neurons=300,batchsize=1,weights=pre_trained_weights,pruning=10,pruning_method='in',stop_function=2,stop_parameter=0.02)
nn_pr_csc.train(TRAINING,TESTING)

for i in [3,5,10,16]:
    pruned_weights = nn_pr_csc.getWeights()
    nn_pr_km = Neural_Network_PR_KM(neurons=300,batchsize=1,cluster=i,pruning=10,pre_weights=pruned_weights,stop_function=2,stop_parameter=0.02)
    nn_pr_km.train(TRAINING,TESTING)

-- Training Session Start (2019-01-23 14:47:27.071659) --

Neurons: 300
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Pruning: 10% (in)
Stop Function: improvements below 0.02%

Epoch 001 (1m0s)   Accuracy TRAIN: 97.56%	Accuracy TEST: 96.81%	Min: 94.65% (7)
Epoch 002 (2m0s)   Accuracy TRAIN: 97.97%	Accuracy TEST: 97.19%	Min: 95.62% (7)
Epoch 003 (2m58s)  Accuracy TRAIN: 98.15%	Accuracy TEST: 97.35%	Min: 95.91% (7)
Epoch 004 (3m57s)  Accuracy TRAIN: 98.26%	Accuracy TEST: 97.49%	Min: 96.11% (7)
Epoch 005 (4m56s)  Accuracy TRAIN: 98.37%	Accuracy TEST: 97.56%	Min: 96.3% (7)
Epoch 006 (5m55s)  Accuracy TRAIN: 98.44%	Accuracy TEST: 97.58%	Min: 96.4% (7)
Epoch 007 (6m54s)  Accuracy TRAIN: 98.5%	Accuracy TEST: 97.59%	Min: 96.4% (7)
Epoch 008 (7m53s)  Accuracy TRAIN: 98.54%	Accuracy TEST: 97.63%	Min: 96.3% (7)
Epoch 009 (8m51s)  Accuracy TRAIN: 98.59%	Accuracy TEST: 97.64%	Min: 96.3% (7)
Epoch 010 (9m50s)  Accuracy TRAIN: 98.63%	Accuracy TEST: 97.66%	Min: 96.3% (7)
Epoch 011 (10m49s) Accura

# 100

In [284]:
nn100 = Neural_Network(neurons=100,batchsize=1,stop_function=2,stop_parameter=0.01)
nn100.train(TRAINING,TESTING)

-- Training Session Start (2019-01-23 16:16:57.578723) --

Neurons: 100
Batch Train: 60000
Batch Test: 10000
Stop Function: improvements below 0.01%

Epoch 001 (17s)    Accuracy TRAIN: 92.84%	Accuracy TEST: 93.04%	Min: 85.65% (5)
Epoch 010 (2m59s)  Accuracy TRAIN: 98.01%	Accuracy TEST: 97.03%	Min: 95.14% (9)
Epoch 020 (5m56s)  Accuracy TRAIN: 98.64%	Accuracy TEST: 97.39%	Min: 95.84% (9)
Epoch 025 (7m24s)  Accuracy TRAIN: 98.77%	Accuracy TEST: 97.39%	Min: 96.13% (9)

-- Training Session End (2019-01-23 16:24:22.539351) --


In [287]:
for i in [32,48,64,96,128,192,256,384,512,768,1024,1536,2048]:
    pre_trained_weights = nn100.getWeights()
    nn_km = Neural_Network_KM(neurons=100,batchsize=1,cluster=i,pre_weights=pre_trained_weights,stop_function=2,stop_parameter=0.02)
    nn_km.train(TRAINING,TESTING)

-- Training Session Start (2019-01-23 18:53:14.628922) --

Neurons: 100
Clusters: 32
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (1m4s)   Accuracy TRAIN: 11.22%	Accuracy TEST: 11.32%	Min: 00.0% (0)
Epoch 002 (2m7s)   Accuracy TRAIN: 11.23%	Accuracy TEST: 11.33%	Min: 00.0% (0)

-- Training Session End (2019-01-23 18:55:22.083035) --
-------------------------
32,12.778,2m7s,11.23,11.33
-------------------------

-- Training Session Start (2019-01-23 18:55:22.806599) --

Neurons: 100
Clusters: 48
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (1m2s)   Accuracy TRAIN: 28.5%	Accuracy TEST: 28.32%	Min: 00.0% (0)
Epoch 002 (2m6s)   Accuracy TRAIN: 28.98%	Accuracy TEST: 28.82%	Min: 00.0% (0)
Epoch 003 (3m8s)   Accuracy TRAIN: 98.68%	Accuracy TEST: 97.16%	Min: 94.65% (9)
Epoch 004 (4m10s)  Accuracy TRAIN: 98.72%	Accuracy TEST: 97.15%	Min: 94.85% (9)
Epoch 005 (5m15s)  Accuracy TRAIN:

/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=384. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-23 19:19:48.533350) --

Neurons: 100
Clusters: 384
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (1m5s)   Accuracy TRAIN: 98.75%	Accuracy TEST: 97.25%	Min: 94.65% (9)
Epoch 002 (2m10s)  Accuracy TRAIN: 98.8%	Accuracy TEST: 97.21%	Min: 94.85% (9)
Epoch 003 (3m15s)  Accuracy TRAIN: 98.83%	Accuracy TEST: 97.3%	Min: 95.04% (9)
Epoch 004 (4m21s)  Accuracy TRAIN: 98.84%	Accuracy TEST: 97.32%	Min: 95.14% (9)

-- Training Session End (2019-01-23 19:24:09.796086) --
-------------------------
384,7.365,4m21s,98.84,97.32
-------------------------



/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-23 19:24:11.789501) --

Neurons: 100
Clusters: 512
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (1m6s)   Accuracy TRAIN: 98.76%	Accuracy TEST: 97.25%	Min: 95.74% (9)
Epoch 002 (2m13s)  Accuracy TRAIN: 98.84%	Accuracy TEST: 97.33%	Min: 95.14% (9)
Epoch 003 (3m20s)  Accuracy TRAIN: 98.86%	Accuracy TEST: 97.36%	Min: 95.24% (9)

-- Training Session End (2019-01-23 19:27:32.588132) --
-------------------------
512,7.003,3m20s,98.86,97.36
-------------------------



/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=768. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-23 19:27:35.383694) --

Neurons: 100
Clusters: 768
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (1m10s)  Accuracy TRAIN: 98.75%	Accuracy TEST: 97.33%	Min: 95.84% (9)
Epoch 002 (2m21s)  Accuracy TRAIN: 98.85%	Accuracy TEST: 97.35%	Min: 95.54% (9)
Epoch 003 (3m31s)  Accuracy TRAIN: 98.88%	Accuracy TEST: 97.37%	Min: 95.64% (9)
Epoch 004 (4m42s)  Accuracy TRAIN: 98.91%	Accuracy TEST: 97.36%	Min: 95.54% (9)
Epoch 005 (5m53s)  Accuracy TRAIN: 98.92%	Accuracy TEST: 97.38%	Min: 95.64% (9)

-- Training Session End (2019-01-23 19:33:28.422393) --
-------------------------
768,6.535,5m53s,98.92,97.38
-------------------------



/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=1024. Setting it to 3*k
  init_size=init_size)


ValueError: Number of samples smaller than number of clusters.

In [288]:
for i in range(60,100,10):
    pre_trained_weights = nn100.getWeights()
    nn_pr_csc = Neural_Network_PR_CSC(neurons=100,batchsize=1,weights=pre_trained_weights,pruning=i,pruning_method='in',stop_function=2,stop_parameter=0.02)
    nn_pr_csc.train(TRAINING,TESTING)

    for j in [32,48,64,96,128,192,256,384,512]:
        pruned_weights = nn_pr_csc.getWeights()
        nn_pr_km = Neural_Network_PR_KM(neurons=100,batchsize=1,cluster=j,pruning=i,pre_weights=pruned_weights,stop_function=2,stop_parameter=0.02)
        nn_pr_km.train(TRAINING,TESTING)

-- Training Session Start (2019-01-23 20:39:24.657169) --

Neurons: 100
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Pruning: 60% (in)
Stop Function: improvements below 0.02%

Epoch 001 (37s)    Accuracy TRAIN: 98.62%	Accuracy TEST: 97.41%	Min: 95.72% (7)
Epoch 002 (1m14s)  Accuracy TRAIN: 98.78%	Accuracy TEST: 97.44%	Min: 96.11% (7)
Epoch 003 (1m52s)  Accuracy TRAIN: 98.88%	Accuracy TEST: 97.49%	Min: 96.23% (9)
Epoch 004 (2m29s)  Accuracy TRAIN: 98.95%	Accuracy TEST: 97.5%	Min: 95.94% (9)
Epoch 005 (3m7s)   Accuracy TRAIN: 99.01%	Accuracy TEST: 97.51%	Min: 95.94% (9)
Epoch 006 (3m44s)  Accuracy TRAIN: 99.04%	Accuracy TEST: 97.49%	Min: 95.84% (9)
Epoch 007 (4m21s)  Accuracy TRAIN: 99.09%	Accuracy TEST: 97.5%	Min: 95.84% (9)
Epoch 008 (4m59s)  Accuracy TRAIN: 99.12%	Accuracy TEST: 97.49%	Min: 95.84% (9)
Epoch 009 (5m36s)  Accuracy TRAIN: 99.14%	Accuracy TEST: 97.52%	Min: 96.04% (9)

-- Training Session End (2019-01-23 20:45:01.309127) --
-------------------------
0.6,1.667,5m36s,9

/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=384. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-23 21:39:56.514881) --

Neurons: 100
Pruning: 0.60
Clusters: 384
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (3m6s)   Accuracy TRAIN: 99.12%	Accuracy TEST: 97.46%	Min: 95.44% (9)
Epoch 002 (6m12s)  Accuracy TRAIN: 99.14%	Accuracy TEST: 97.43%	Min: 95.54% (9)

-- Training Session End (2019-01-23 21:46:09.275219) --
-------------------------
0.6,384,12.178,6m12s,99.14,97.43
-------------------------



/Users/Moris/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


-- Training Session Start (2019-01-23 21:46:10.836734) --

Neurons: 100
Pruning: 0.60
Clusters: 512
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (3m7s)   Accuracy TRAIN: 99.14%	Accuracy TEST: 97.38%	Min: 95.74% (9)
Epoch 002 (6m14s)  Accuracy TRAIN: 99.16%	Accuracy TEST: 97.41%	Min: 95.64% (9)

-- Training Session End (2019-01-23 21:52:25.336038) --
-------------------------
0.6,512,11.554,6m14s,99.16,97.41
-------------------------

-- Training Session Start (2019-01-23 21:52:25.369644) --

Neurons: 100
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Pruning: 70% (in)
Stop Function: improvements below 0.02%

Epoch 001 (38s)    Accuracy TRAIN: 98.6%	Accuracy TEST: 97.38%	Min: 95.53% (7)
Epoch 002 (1m17s)  Accuracy TRAIN: 98.78%	Accuracy TEST: 97.53%	Min: 96.11% (7)
Epoch 003 (1m55s)  Accuracy TRAIN: 98.89%	Accuracy TEST: 97.56%	Min: 96.21% (7)
Epoch 004 (2m34s)  Accuracy TRAIN: 98.97%	Accuracy TEST: 97.54%	Min: 96.23% (9)
Epoc

-- Training Session Start (2019-01-23 23:41:48.044974) --

Neurons: 100
Pruning: 0.80
Clusters: 48
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (3m36s)  Accuracy TRAIN: 98.74%	Accuracy TEST: 96.76%	Min: 93.95% (9)
Epoch 002 (7m15s)  Accuracy TRAIN: 98.92%	Accuracy TEST: 97.07%	Min: 94.55% (9)
Epoch 003 (10m50s) Accuracy TRAIN: 98.95%	Accuracy TEST: 97.08%	Min: 94.45% (9)
Epoch 004 (14m26s) Accuracy TRAIN: 98.94%	Accuracy TEST: 97.15%	Min: 94.65% (9)

-- Training Session End (2019-01-23 23:56:14.354980) --
-------------------------
0.8,48,14.282,14m26s,98.94,97.15
-------------------------

-- Training Session Start (2019-01-23 23:56:14.842736) --

Neurons: 100
Pruning: 0.80
Clusters: 64
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (3m35s)  Accuracy TRAIN: 98.88%	Accuracy TEST: 97.19%	Min: 95.34% (9)
Epoch 002 (7m10s)  Accuracy TRAIN: 98.92%	Accuracy TEST: 97.16%	Min: 95.14%

Epoch 001 (3m38s)  Accuracy TRAIN: 99.05%	Accuracy TEST: 97.36%	Min: 95.44% (9)
Epoch 002 (7m16s)  Accuracy TRAIN: 99.1%	Accuracy TEST: 97.28%	Min: 94.95% (9)
Epoch 003 (10m55s) Accuracy TRAIN: 99.12%	Accuracy TEST: 97.44%	Min: 95.34% (9)
Epoch 004 (14m33s) Accuracy TRAIN: 99.13%	Accuracy TEST: 97.46%	Min: 95.14% (9)

-- Training Session End (2019-01-24 02:19:40.815446) --
-------------------------
0.9,128,10.103,14m33s,99.13,97.46
-------------------------

-- Training Session Start (2019-01-24 02:19:41.664084) --

Neurons: 100
Pruning: 0.90
Clusters: 192
Batch Train: 60000 (100%)
Batch Test: 10000 (100%)
Stop Function: improvements below 0.02%

Epoch 001 (3m39s)  Accuracy TRAIN: 99.0%	Accuracy TEST: 97.32%	Min: 95.54% (9)
Epoch 002 (7m17s)  Accuracy TRAIN: 99.12%	Accuracy TEST: 97.46%	Min: 95.34% (9)
Epoch 003 (10m56s) Accuracy TRAIN: 99.15%	Accuracy TEST: 97.48%	Min: 95.24% (9)
Epoch 004 (14m35s) Accuracy TRAIN: 99.15%	Accuracy TEST: 97.54%	Min: 95.04% (9)

-- Training Session End (

# RESULT

In [202]:
def read_csv(file_name):
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        header = next(csv_reader, None)
        data = np.array([[t for t in row] for row in csv_reader])
        return header,data.T

In [ ]:
p = read_csv('experiments/csv/pruning.csv')
plt.figure(figsize=(16,8))
plt.plot(p[1][0],p[1][4])
plt.plot(p[1][0],p[1][3])
plt.show()

In [249]:
pc = read_csv('experiments/pruning_clustering.csv')